In [1]:
import config
import pandas as pd
from classrad.utils import io
from classrad.utils.statistics import describe_auc, describe_stat, describe_f1
from classrad.evaluation.utils import get_youden_threshold
import numpy as np

tasks = ["prostate-ucla", "prostatex"]
rois = ["prostate", "lesion"]
params = config.PARAMS

for task in tasks:
    for roi in rois:
        result_dir = config.RESULT_DIR / (task) / roi
        results_internal = io.load_json(result_dir / "internal_test.json")
        results_external = io.load_json(result_dir / "external_test.json")
        df = pd.read_csv(result_dir / "internal_test.csv")
        df_external = pd.read_csv(result_dir / "external_test.csv")
        print(f"{task}_{roi}:")
        auc_mean_cv = np.mean(results_internal["AUC train"])
        auc_std_cv = np.std(results_internal["AUC train"])
        print(f"AUC train: {auc_mean_cv:.3f} +/- {auc_std_cv:.3f}")
        auc_mean, auc_lower, auc_upper = describe_auc(df['y_true'], df['y_pred'])
        thr, _, _ = get_youden_threshold(df['y_true'], df['y_pred'])
        f1_tuple = describe_f1(df['y_true'], (df['y_pred'] >= thr))
        sens_tuple, spec_tuple = describe_stat(df['y_true'], (df['y_pred'] >= thr))
        # print(f"AUC test: {auc_mean:.3f} [{auc_lower:.3f}, {auc_upper:.3f}]")
        print("External:")
        auc_mean_ext, auc_lower_ext, auc_upper_ext = describe_auc(df_external['y_true'], df_external['y_pred'])
        auc_ext_test = results_external["AUC test"]
        thr, _, _ = get_youden_threshold(df_external['y_true'], df_external['y_pred'])
        # print(f"AUC test ext: {auc_ext_test:.3f} [{auc_lower_ext:.3f}, {auc_upper_ext:.3f}]")
        f1_tuple = describe_f1(df_external['y_true'], (df_external['y_pred'] > thr))
        sens_tuple, spec_tuple = describe_stat(df_external['y_true'], (df_external['y_pred'] > thr))

prostate-ucla_prostate:
AUC train: 0.475 +/- 0.072
Mean AUC=0.39371264275540024, 95% CI [0.29072445129142505, 0.5015635530327343]
F1 score: Mean=0.1345241073171418, 95% CI [0.047045143638850886,           0.23160342717258262]
Sensitivity: Mean=0.07280944838983064, 95% CI [0.024089213998852554,           0.13096804511278196]
Specificity: Mean=1.0, 95% CI [1.0,           1.0]
External:
Mean AUC=0.47803053140809143, 95% CI [0.14038318452380955, 0.8095982142857142]
F1 score: Mean=0.6585129245650828, 95% CI [0.39999999999999997,           0.8666666666666666]
Sensitivity: Mean=0.5729946055922062, 95% CI [0.30750000000000005,           0.8333333333333334]
Specificity: Mean=0.6649614690864691, 95% CI [0.2,           1.0]
prostate-ucla_lesion:
AUC train: 0.559 +/- 0.045
Mean AUC=0.5843409655362795, 95% CI [0.46192193223443223, 0.7006811019263126]
F1 score: Mean=0.6169819656193147, 95% CI [0.5202076981707318,           0.7092312971859984]
Sensitivity: Mean=0.5072943084769967, 95% CI [0.397590361

In [4]:
import utils

for task in tasks:
    result_dir = config.RESULT_DIR / task / "prostate"
    df = pd.read_csv(result_dir / "prostate_features.csv")
    df_prepro = utils.add_label_col(df)
    split_path = result_dir / "splits.json"
    splits = io.load_json(split_path)
    split_on = params[task]["split_on"]
    df_prepro = df_prepro[[split_on, "label"]].drop_duplicates()
    df_prepro['test'] = 0
    df_prepro.loc[df_prepro[split_on].isin(splits["test"]), "test"] = 1
    print(df_prepro.groupby(by=["test", "label"]).count())

            patient_ID
test label            
0    False         142
     True          479
1    False          36
     True          120
            case_ID
test label         
0    False       80
     True        78
1    False       20
     True        20


In [3]:
splits

{'split_type': 'test + cross-validation on the training',
 'test': ['ProstateX-0073',
  'ProstateX-0174',
  'ProstateX-0016',
  'ProstateX-0113',
  'ProstateX-0059',
  'ProstateX-0156',
  'ProstateX-0000',
  'ProstateX-0152',
  'ProstateX-0075',
  'ProstateX-0102',
  'ProstateX-0072',
  'ProstateX-0092',
  'ProstateX-0042',
  'ProstateX-0094',
  'ProstateX-0153',
  'ProstateX-0170',
  'ProstateX-0087',
  'ProstateX-0135',
  'ProstateX-0089',
  'ProstateX-0147',
  'ProstateX-0050',
  'ProstateX-0122',
  'ProstateX-0040',
  'ProstateX-0187',
  'ProstateX-0177',
  'ProstateX-0116',
  'ProstateX-0112',
  'ProstateX-0117',
  'ProstateX-0007',
  'ProstateX-0148',
  'ProstateX-0159',
  'ProstateX-0029',
  'ProstateX-0151',
  'ProstateX-0136',
  'ProstateX-0001',
  'ProstateX-0139',
  'ProstateX-0124',
  'ProstateX-0077',
  'ProstateX-0109',
  'ProstateX-0110'],
 'train': {'fold_0': [['ProstateX-0179',
    'ProstateX-0074',
    'ProstateX-0024',
    'ProstateX-0083',
    'ProstateX-0037',
    